## Contains a multi-quantile implementation of the sBQR loss

In [2]:
import numpy as np
import pandas as pd
import math
import os
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from adahessian import Adahessian, get_params_grad
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
def barePDF(x, tau):
    # x is a torch tensor, tau is a float
    ind= (torch.sign(x)+1)/2 # mask about the origin
    quantFactor= (1-tau)*(1-ind) + tau*ind
    return 2/math.pi*torch.cosh(x).pow_(-1)*quantFactor

def bareCDF(yhat, tau):
    # yhat is a torch tensor, tau is a float
    ind= (torch.sign(yhat)+1)/2 # mask about the origin
    quantFactor= (1-tau)*(1-ind) + tau*ind
    return tau+4*quantFactor/math.pi*torch.atan(torch.tanh(yhat/2))

def quantilePDF(x, qs):
    # x is a torch tensor, qs is a list of floats
    quantilePDs= []
    for q in qs:
        quantilePDs.append(barePDF(x, q))
    return quantilePDs

def quantileCDF(yhat, qs):
    # yhat is a torch tensor, qs is a list of floats
    quantileCDs= []
    for q in qs:
        quantileCDs.append(bareCDF(yhat, q))
    return quantileCDs 

def baresBQR(y, yhat, tau):
    # y and yhat are torch tensors, tau is a float
    return y*torch.log(1-bareCDF(yhat, tau))+(1-y)*torch.log(bareCDF(yhat, tau))

def sBQR(y, yhat, qs):
    # y and yhat are torch tensors and qs is a list of floats
    quantilesBQRs= []
    for q in qs:
        quantilesBQRs.append(baresBQR(y, yhat, q))
    return quantilesBQRs

class sBQRL(nn.Module):
    def __init__(self):
        super(sBQRL, self).__init__()
    
    def forward(self, y, yhat, qs):
        return torch.mean(torch.cat(sBQR(y, yhat, qs)))